# Preliminaries

In [23]:
import math
from sympy import symbols, sqrt, gcd, Integer, latex, floor, together, Add
from IPython.display import display, Math


#Use together to recombine any sums of fractions and keep them as a single fraction.

def latex_fraction_compact(expr, parentheses_for_sum=True):
    expr = together(expr)  # combina in una sola frazione
    num, den = expr.as_numer_denom()
    num_ltx = latex(num)
    if parentheses_for_sum and num.is_Add:
        num_ltx =  num_ltx 
    return r"\frac{" + num_ltx + '}{' + latex(den) + "}"

# Functions

In [24]:
def is_perfect_square(n):
    #Checks whether an integer (positive or negative) is a perfect square. If it is negative, returns False
    if n < 0:
        return False
    
    root = int(math.isqrt(n))

    if root * root == n:
        return True
    return False

## Polinomi

In [25]:
def analizza_polinomio(A, B, C):
#Analyzes the quadratic polynomial Ax^2 + Bx + C and checks that the coefficients are coprime,
#that the discriminant is positive and not a perfect square. Returns the discriminant if valid; otherwise shows error messages


    # step 1: Verifying the coefficients ---
    # Verifying coprimality
    comun_div = math.gcd(A, math.gcd(B, C))

    if comun_div != 1:
        display(Math(r"\text{ERROR: The coefficients are not coprime. GCD} = " + str(comun_div)))
        return

    # Step 2: Analyzing the Discriminant
    delta = B**2 - 4*A*C 
    # delta is an integer since A, B, C are integers
    
    display(Math(  r"\text{Analysis of the polynomial: } " 
    + latex(A) + r"x^2 + (" 
    + latex(B) + r")x + (" 
    + latex(C) + r")"))


    display(Math(r"\text{Discriminant } \Delta = " + latex(delta)))

    if delta <= 0:
        display(Math(r"\text{ERROR: The discriminant must be positive and non-zero.}"))
        return
    
    if is_perfect_square(delta):

        display(Math(r"\text{ERROR: The discriminant is a perfect square (}" + latex(delta) + r" = " + latex(sqrt(delta)) + r"^2)" + r"\text{. The roots would be rational.}"))
        return

    display(Math(r"\text{OK: Valid polynomial (coprime coefficients, } \Delta > 0 \text{, not a perfect square).}"))
    print("-" * 30)

    return delta

In [26]:
def calcola_radici(A, B, delta):
# Compute the roots of the quadratic polynomial A*x^2 + B*x + C given the discriminant delta. 

# Returns the symbolic roots both written with +sqrt(delta) 
    # We use Sympy's Integer to ensure they are treated as symbolic objects
    sym_A = Integer(A)
    sym_B = Integer(B)
    sym_delta = Integer(delta)

    # We construct the roots avoiding transforming them into sums of fractions.
    # Add(..., evaluate=False) prevents combinations that could distribute the denominator.
    num1 = Add(-sym_B, sqrt(sym_delta), evaluate=False)
    num2 = Add(sym_B, sqrt(sym_delta), evaluate=False)
    alpha1 = num1 / (2 * sym_A)
    alpha2 = num2 / (-2 * sym_A)

    display(Math(r"\text{The roots are: }"))
    display(Math(r"\alpha_1 = " + latex_fraction_compact(alpha1)))
    display(Math(r"\alpha_2 = " + latex_fraction_compact(alpha2)))
    print("-" * 30)

    return alpha1, alpha2

## Frazioni Continuate

In [27]:
def get_continued_fraction_periodic_and_tails(P, D, Q):
    """
    Calculates the continued fraction of a quadratic irrational of the form (P + sqrt(D)) / Q.
    Returns the list of pre-period coefficients and the list of the period.
    Algorithm based on integer arithmetic to avoid rounding errors.
    """
    # 1. Calculate the first coefficient a0
    # Note: math.isqrt(D) is the integer part of the square root of D

    a0 = floor((P + sqrt(D)) / Q)

    INT_P = Integer(P)
    INT_Q = Integer(Q)
    INT_D = Integer(D)
    alpha0 = (INT_P+ sqrt(INT_D)) / (INT_Q)
  
    
    coeff_list = [a0]
    tails_list = [alpha0]

    # We store the triples (P, Q, a) to detect the cycle (period)
    history = {}
    
    # Initial variables for the iteration
    p = int(P)
    q = int(Q)
    a = int(a0)
    alpha=alpha0

    idx = 0
    history[(p, q)] = idx
    
    while True:
        # Recurrence formulas for continued fractions of quadratic irrationals
        p =int(q * a - p)
        q =int((D - p**2) // q)
        a =floor((p + sqrt(D)) / q)


        INT_P = Integer(p)
        INT_Q = Integer(q)     
        alpha = (INT_P + sqrt(INT_D)) / (INT_Q)

        # If we have already seen this pair (p, q), we have found the start of the period
        if (p, q) in history:
            start_period_idx = history[(p, q)]

            if start_period_idx == 0:
                # Case where there is no pre-period, everything is period
                pre_period = 0 
                period = coeff_list[:] # this command takes all elements of the list
                tails = tails_list[:] # this command takes all elements of the list tails_list
                return pre_period, period, tails
        

            pre_period = coeff_list[:start_period_idx] # this command takes all elements of the list before start_period_idx 
            period = coeff_list[start_period_idx:] # this command takes all elements of the list after start_period_idx until the end
            tails = tails_list[:] # this command takes all elements of the list tails_list

            return pre_period, period, tails
        
        coeff_list.append(a)
        tails_list.append(alpha)
        idx += 1
        history[(p, q)] = idx

In [28]:
def print_continued_fraction(pre_period, period, tails):
    """
    Print the continued fraction in a readable format.
    """
    # create the period part as "a, b, c"
    period_body = ", ".join(map(str, period))

    if pre_period == 0:
        # no pre-period: [ \overline{period} ]
        cf_str = r"[" + r"\overline{" + period_body + r"}]"
    else:
        pre_body = ", ".join(map(str, pre_period))
        cf_str = r"[" + pre_body + r"; " + r"\overline{" + period_body + r"}]"

    # display alpha_0 := continued fraction
    display(Math(latex_fraction_compact(tails[0]) + r":= " + cf_str))

    display(Math(r"\text{Successive tails: }"))
    for i, tail in enumerate(tails):
        display(Math(r"\alpha_{" + str(i) + r"} = " + latex_fraction_compact(tail)))



# Codice Intero

In [29]:
def from_polinomial_to_continued_fraction(A, B, C):
    # Compute the continued fraction of the roots with positive discriminant of the quadratic polynomial A*x^2 + B*x + C

    delta = analizza_polinomio(A, B, C)
    if delta is None:
        return          
    
    alpha1, alpha2 = calcola_radici(A, B, delta)   


    # Compute the continued fraction for alpha1

    P1 = -B
    Q1 = 2 * A

    pre_period1, period1, tails1 = get_continued_fraction_periodic_and_tails(P1, delta, Q1) 

    print_continued_fraction(pre_period1, period1, tails1)

    # Compute the continued fraction for alpha2
    P2 = B
    Q2 = -2 * A

    pre_period2, period2, tails2 = get_continued_fraction_periodic_and_tails(P2, delta, Q2) 

    print_continued_fraction(pre_period2, period2, tails2)
    return


In [30]:
from_polinomial_to_continued_fraction(1628,-3876,2307)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

------------------------------


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>